In [1]:
from pathlib import Path
import os

yr = 2023
d = 19

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  formatted_input = {'workflows': {}, 'parts': []}
  done_with_workflows = False
  for l in inp.splitlines():
    if len(l) == 0:
      done_with_workflows = True
      continue
    if not done_with_workflows:
      workflow_name = l.split('{')[0]
      workflow_info = l.split('{')[1].replace('}','')
      workflow_instructions = workflow_info.split(',')
      cur_workflow_tests = []
      for inst in workflow_instructions:
        if '>' in inst:
          comp = '>'
          c = inst.split('>')[0]
          val = int(inst.split('>')[1].split(':')[0])
          dest = inst.split('>')[1].split(':')[1]
          workflow_func = lambda x, c=c, val=val: x[c]>val
        elif '<' in inst:
          comp = '<'
          c = inst.split('<')[0]
          val = int(inst.split('<')[1].split(':')[0])
          dest = inst.split('<')[1].split(':')[1]
          workflow_func = lambda x, c=c, val=val: x[c]<val
        else:
          c = None
          comp = None
          val = None
          dest = inst
          workflow_func = lambda x: True
        cur_workflow_tests.append({'test': workflow_func,
                                   'dest': dest,
                                   'c': c,
                                   'comp': comp,
                                   'val': val})
      formatted_input['workflows'][workflow_name] = cur_workflow_tests
    else:
      part = {s.split('=')[0]: int(s.split('=')[1])
              for s in l.replace('}','').replace('{','').split(',')}
      formatted_input['parts'].append(part)
  return formatted_input

In [3]:
def process_part(workflows, part, debug=False):
  '''
  Workflows are saved as lists of dictionaries
  containing lambda functions that perform the test
  as well as destination values which denote
  where the part will be sent to upon a positive
  test result (this will be either 'A', 'R', or
  another workflow). Starting at the 'in'
  workflow, send the part through the workflows until
  we arrive at 'A', or 'R'
  '''
  def apply_workflow(workflow, part):
    for w in workflow:
      if w['test'](part):
        return w['dest']

  res = apply_workflow(workflows['in'], part)
  while res not in ['A', 'R']:
    res = apply_workflow(workflows[res], part)
  return res=='A'

In [4]:
def sum_of_accepted(formatted_input):
  '''
  Just process each part and add up their category values
  '''
  s = 0
  for p in formatted_input['parts']:
    if process_part(formatted_input['workflows'], p):
      s += sum(p.values())
  return s

In [5]:
def find_distinct_combos(formatted_input):
  '''
  Start with the full possible ranges (1-4000 for all 4 categories)
  Work through the workflows recursively:
    To process one workflow loop over the tests within each workflow
    in order keeping a rolling sum of potential combos.
    At each test IF the test sends you to 'A', add the total number of combos
    yielded by the current possible ranges that trigger that test. IF the test
    sends you to 'R', add 0 to the sum since those are not accepted combos, IF
    the current test sends you to another workflow, then add the result of
    recursing on that workflow to our rolling sum with only the ranges from our
    current range that trigger that workflow.
    At the end of each iteration of the test loop, set the current ranges that
    we are examining to only include those ranges that are NOT triggered by
    the previous test.
  '''
  import numpy as np
  from copy import deepcopy

  def trim_range(test, ranges, negate=False):
    '''
    Given a dictionary of ranges trim those ranges to either contain only the
    ranges that trigger the current test, or (if negate==True) only
    the ranges that do NOT trigger the current test
    '''
    ranges = deepcopy(ranges)

    r = test['c']
    if r is None:
      return ranges

    val = test['val']
    comp = test['comp']
    cur_r = ranges[r]
    if comp == '>':
      cur_r = (val+1, cur_r[1]) if not negate else (cur_r[0], val)
    elif comp == '<':
      cur_r = (cur_r[0], val-1) if not negate else (val, cur_r[1])
    else:
      raise Exception('Trying to Trim with an Invalid Comparison')
    ranges[r] = cur_r
    return ranges

  def count_of_ranges(ranges):
    '''
    Get the total number of combos yielded by a specific set of ranges
    '''
    return np.prod(np.array([(ranges[r][1]-ranges[r][0]+1) for r in 'xmas']), dtype=np.ulonglong)


  def _find_distinct_combos(workflows, cur_workflow, cur_ranges):
    '''
    This is the function that we will recurse over
    '''

    # Not sure if we need this but just to be safe
    if any([(cur_ranges[r][1]-cur_ranges[r][0])<0 for r in 'xmas']):
      return 0

    # Just doing this to avoid overflows on our sum
    s = np.array([0], dtype=np.ulonglong)
    for t in workflows[cur_workflow]:
      if t['dest'] == 'A':
        # Add the number of combos in the given ranges and trim our ranges
        s[0] += count_of_ranges(trim_range(t, cur_ranges))
      elif t['dest'] == 'R':
        # These aren't valid combos so just pass
        s[0] += 0
      else:
        # Add the result of the recursion
        s[0] += _find_distinct_combos(workflows,
                                   cur_workflow=t['dest'],
                                   cur_ranges=trim_range(t, cur_ranges))

      # If we made it to the current test in the workflow
      # then we need to adjust our ranges to exclude combos that
      # got caught by the previous test
      cur_ranges = trim_range(t, cur_ranges, negate=True)
    return s[0]


  workflows = formatted_input['workflows']

  return _find_distinct_combos(workflows,
                               cur_workflow = 'in',
                               cur_ranges = {'x':(1,4000),
                                            'm':(1,4000),
                                            'a':(1,4000),
                                            's':(1,4000)}
                              )

In [6]:
import time

t = time.time()

formatted_input = format_input(inp)

print(sum_of_accepted(formatted_input))
print(find_distinct_combos(formatted_input))


print('\nRUNTIME: ', time.time()-t)

263678
125455345557345

RUNTIME:  0.10571646690368652
